<a href="https://colab.research.google.com/github/KairaNithin/TPU-Repos/blob/main/Pytorch_Lightining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl
!pip install pytorch-lightning
import pytorch_lightning as pl

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
# train_loader = torch.load("drive/MyDrive/train_dataloader.pth")
train_loader = torch.load("drive/MyDrive/XLM_512.pth")

In [4]:
!pip install sentence_transformers
!pip install transformers
from sentence_transformers import SentenceTransformer

In [5]:
from transformers import XLMRobertaTokenizer, XLMRobertaModel

In [6]:
class ClassificationHead(pl.LightningModule):
  """ Classification head for the Roberta Model """ 
  def __init__(self, numberOfClasses, hidden_size_bert, hidden_size_post_feats, dropout_val = 0.2):
    super().__init__()
    self.denseInit = nn.Linear(hidden_size_post_feats, hidden_size_bert)
    self.dense = nn.Linear(hidden_size_bert, hidden_size_bert)
    self.dropout = nn.Dropout(dropout_val)
    self.output = nn.Linear(hidden_size_bert, numberOfClasses)
  def forward(self, x):
    # print(x.shape)
    x = self.dropout(x)
    x = self.denseInit(x)
    x = torch.tanh(x)
    x = self.dropout(x)
    x = self.dense(x)
    x  = torch.tanh(x)
    x = self.dropout(x)
    x  = self.output(x)
    return x

class TextClassification(pl.LightningModule):
  """ Classifier with feature injection """
  def __init__(self, numberOfClasses,dropout_val = 0.1, batch_size = 16):
    super(TextClassification, self).__init__()
    self.bert = XLMRobertaModel.from_pretrained("sentence-transformers/xlm-r-100langs-bert-base-nli-mean-tokens")
    self.classifier = ClassificationHead(numberOfClasses, self.bert.config.hidden_size, (self.bert.config.hidden_size * 2 + 300) , dropout_val)
  def forward(self, input_seq, attention_mask, emoji, hashTag):
    bert_pooled_output = self.bert(input_seq, attention_mask=attention_mask)[0]
    bert_pooled_output = bert_pooled_output[:, 0, :]
    bert_pooled_out_feat = torch.cat([bert_pooled_output, emoji, hashTag], axis = 1)
    # print("Shape",bert_pooled_out_feat.shape)
    output = self.classifier(bert_pooled_out_feat)
    return output

  def configure_optimizers(self):
    optimizer = torch.optim.AdamW(self.parameters(), lr=2e-5, eps=1e-8)
    return optimizer

  def training_step(self, train_batch, batch_idx):

    b_input_ids = train_batch[1]
    b_input_mask = train_batch[2]
    b_labels = train_batch[6]
    b_emoji = train_batch[5]
    b_hashtag = train_batch[4]
    loss_function = nn.CrossEntropyLoss()

    pred = self.forward(b_input_ids,b_input_mask ,b_emoji.float(), b_hashtag.float())
    loss = loss_function(pred.view(-1, 2), b_labels.view(-1))

    # x, y = train_batch
    # x = x.view(x.size(0), -1)
    # z = self.encoder(x)    
    # x_hat = self.decoder(z)
    # loss = F.mse_loss(x_hat, x)
    self.log('train_loss', loss)
    return loss

In [11]:
import torch.nn as nn
model = TextClassification(2)

In [13]:
for params in model.bert.parameters():
  params.requires_grad = False

In [14]:
trainer = pl.Trainer(tpu_cores=8)
trainer.fit(model,train_loader)

GPU available: False, used: False
TPU available: True, using: 8 TPU cores

  | Name       | Type               | Params
--------------------------------------------------
0 | bert       | XLMRobertaModel    | 278 M 
1 | classifier | ClassificationHead | 2.0 M 
--------------------------------------------------
2.0 M     Trainable params
278 M     Non-trainable params
280 M     Total params
1,120.186 Total estimated model params size (MB)


ProcessExitedException: ignored